In [1]:
source("~/sc-online/extraGenes.R")
source("~/sc-online/myExpSetCreatorFn.R")
source("~/sc-online/utils.R")

library("DropletUtils")
library(dplyr)
library(lisi)
library(rhdf5)
library(Seurat)
library(tidyr)


# This magic number is extremely important; 
# it's the probability above which we will take the max of Vireo's predictions of the likely donor 
DONOR_ASSIGNMENT_CUTOFF = 0.3

SAVE_H5_TO_RDS=FALSE

base_path = "~/seq_data/pd"
mol_path  = "outs/molecule_info.h5"
umi_path = "outs/filtered_feature_bc_matrix.h5"
donor_ids_path = "donor_ids.tsv"
prob_singlet_path = "prob_singlet.tsv"
prob_doublet_path = "prob_doublet.tsv"


Loading required package: EnsDb.Hsapiens.v86

Loading required package: ensembldb

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: Genomi

In [2]:
expt_log = read.csv('~/expt_log.csv')
sort(names(expt_log))

[1] "bam_path"                                                                                                                     
 [2] "BCLPath"                                                                                                                      
 [3] "BICCN.ALREADY.UPLOADED"                                                                                                       
 [4] "BICCN.TO.UPLOAD"                                                                                                              
 [5] "chemistry_10x"                                                                                                                
 [6] "date.YYMMDD."                                                                                                                 
 [7] "demultiplex"                                                                                                                  
 [8] "donor_id"                                                                                                                     
 [9] "donor_list"                                                                                                                   
[10] "email"                                                                                                                        
[11] "Estimated.Number..of.Cells"                                                                                                   
[12] "FACS"                                                                                                                         
[13] "fastq_path"                                                                                                                   
[14] "flowcell"                                                                                                                     
[15] "Fraction.Reads.in.Cells"                                                                                                      
[16] "genotype_VCF_path"                                                                                                            
[17] "IlluminaPlatform"                                                                                                             
[18] "Index"                                                                                                                        
[19] "Lane"                                                                                                                         
[20] "library"                                                                                                                      
[21] "library_path"                                                                                                                 
[22] "Matches.library.name.formula"                                                                                                 
[23] "Mean.Reads..per.Cell"                                                                                                         
[24] "Median.Genes..per.Cell"                                                                                                       
[25] "Median.UMI.Counts.per.Cell"                                                                                                   
[26] "Nuclei.Extraction.Protocol"                                                                                                   
[27] "num_expected_cells"                                                                                                           
[28] "Number.of..Reads"                                                                                                             
[29] "Pass_qc"                                                                                                                      
[30] "PlateID"                                                                                                                      
[31] "proje

In [3]:
# only keep project names that are either PD or CalicoPD
expt_log_pd = expt_log[expt_log$project_name %in% c('PD', 'CalicoPD'),]

# Mehrdad started collecting data on 221126
expt_log_pd = expt_log_pd[expt_log_pd$date.YYMMDD. > 221125,]

# Homo Sapiens
expt_log_pd_human = expt_log_pd[expt_log_pd$species == 'HS',]

# Substantia Nigra
expt_log_pd_human_sn = expt_log_pd_human[expt_log_pd_human$region == 'SN',]

#subset columns
cols2keep = c(
    'library',
    'region',
    'species',
    'sub_region.tube_id',
    'donor_id',
    'date.YYMMDD.',
    'flowcell',
    'BCLPath',
    'IlluminaPlatform',
    'num_expected_cells',
    'transcriptome',
    'demultiplex',
    'genotype_VCF_path',
    'donor_list',
    'Lane'
)
expt_log_pd_human_sn = expt_log_pd_human_sn[,cols2keep]

# keep experiments that HAVE been demultiplexed
expt_log_pd_human_sn_demult = expt_log_pd_human_sn[expt_log_pd_human_sn$demultiplex == TRUE,]
dim(expt_log_pd_human_sn_demult)

[1] 49 15

In [4]:
# of these experiments, want to analyze those with both molecular counts and with demultiplexing data
find_outs_command = "find ~/seq_data/pd -mindepth 2 -maxdepth 2 -type d -name outs -exec dirname {} \\;"
counts_list = system(find_outs_command, intern=T)

find_demult_command = "find ~/seq_data/pd -mindepth 2 -maxdepth 2 -type f -name prob_doublet.tsv -exec dirname {} \\;"
demult_list = system(find_demult_command, intern=T)

# get the intersection of the two lists
counts_and_demult_list = basename(counts_list[basename(counts_list) %in% basename(demult_list)])

# only keep experiments with both 
expt_log_pd_human_sn_demult = expt_log_pd_human_sn_demult[expt_log_pd_human_sn_demult$library %in% counts_and_demult_list,]
dim(expt_log_pd_human_sn_demult)

[1] 17 15

In [5]:
if(SAVE_H5_TO_RDS){
    for(name in expt_log_pd_human_sn_demult$library){
        print(name)
        mol_file = file.path(base_path, name, mol_path)
        umi_file = file.path(base_path, name, umi_path)
        donor_ids_file = file.path(base_path, name, donor_ids_path)
        prob_singlet_file = file.path(base_path, name, prob_singlet_path)
        prob_doublet_file = file.path(base_path, name, prob_doublet_path)
        
        # read in the data and save as RDS
        mol_data = read10xMolInfo(mol_file)
        saveRDS(mol_data, file=paste(base_path, name, "outs/molecule_info.rds", sep='/'))

        umi_data = Read10X_h5(umi_file)
        saveRDS(umi_data, file=paste(base_path, name, "outs/filtered_feature_bc_matrix.rds", sep='/'))
        print(" ")
    }
}


In [6]:
# prob_doublet does not have the same number of columns in the data as it does in the header...
# prob_doublet = read.table(
#     file.path(base_path, expt_log_pd_human_sn_demult$library[[1]], prob_doublet_path), 
#     header = TRUE, 
#     sep = "\t")

In [7]:
createMetaDataFromDCGMatrix=function(
    expt_df,
    umi_dcg, 
    mol_dcg,
    donor_ids,
    cutoff=DONOR_ASSIGNMENT_CUTOFF
    ){
    # takes in expt_df, a row of a dataframe 
    # umi_dcg, a dcg matrix derived from a Read 10X filtered feature matrix h5 file
    # mol_dcg, a dcg matrix derived from a Read 10X molecule info h5 file
    # donor_ids, a dataframe derived from vireo's donor assignment tsv file
    # and cutoff, a float between 0 and 1, which is the cutoff for assigning a cell to a donor

    # outputs a dataframe, whose rownames are the colnames of umi_dcg
    
    # output containing columns:
    #   nUMI: total number of UMIs within a cell
    #   nGene: number of unique genes expressed within a cell
    #   nRead: number of total reads within a cell
    #   donor_id: str, either the ID of the donor as specified in the donor_ids.tsv file, or 'unassigned'
    #   prob_donor: float, the probability that the cell is from the donor specified in donor_id
    #   prob_doublet: probability that the droplet represents a doublet, as determined by vireo
    #   prob_singlet: 1-prob_doublet

    #   TODO cols
    #   dataset: str, name of dataset
    #   orig.ident: class of data being loaded
    #   anno_col_map: dict[str: list-of-str] for columns to be added. Keys must start with 'anno_'

    umi_dcg_rownames = rownames(umi_dcg)
    umi_dcg_colnames = colnames(umi_dcg)

    # add a column to donor_ids that is the donor id as determined by the custom cutoff
    donor_ids['donor_id_default'] = donor_ids['donor_id']
    donor_ids['donor_id'] = 'unassigned'
    # when the probability is above our custom cutoff, choose the best singlet
    donor_ids$donor_id[donor_ids$prob_max >= cutoff] = (
        donor_ids$best_singlet[donor_ids$prob_max >= cutoff])

    mol_dcg_matched = mol_dcg[umi_dcg_rownames, umi_dcg_colnames]    
    donor_ids_matched = donor_ids[umi_dcg_colnames,]

    # sum within cells
    nUMI = colSums(umi_dcg)
    nGene = colSums(umi_dcg > 0)
    nRead = colSums(mol_dcg_matched)

    meta_df = data.frame(
        expt_df$library,
        expt_df$region,
        expt_df$species,
        expt_df$date.YYMMDD.,
        expt_df$sub_region.tube_id,
        expt_df$flowcell,
        expt_df$IlluminaPlatform,
        expt_df$Lane,
        expt_df$num_expected_cells,
        expt_df$transcriptome,
        nUMI,
        nGene, 
        nRead,
        donor_ids_matched$donor_id, 
        donor_ids_matched$prob_max,
        1-donor_ids_matched$prob_doublet,
        donor_ids_matched$prob_doublet
    )

    # lowercase colnames and convert to snake_case
    colnames(meta_df) = c(
        "library", 'region', 'species', 'date', 'sub_region', 'flowcell', 'illumina_platform', 
        'lane', 'num_expected_cells', 'transcriptome',
        "nUMI", "nGene", 'nRead', 'donor_id', 'prob_donor', 'prob_singlet', 'prob_doublet')
    rownames(meta_df) = colnames(umi_dcg)
    return(meta_df)
}

In [201]:
# for every experiment, we want to 
# 1. load the `tsv`s mapping from cells to donors
# 2. read the molecule_info and umi data rds
# 3. Generate col metadata
# 4. create a singleCellExperiment object
# 5. save the sce object as an rds file

# we will be inner joining the gns to the mol_data pivoted wide using ensembl_gene_id
# then we will left join the umi_data to the result above using gene symbol
# TODO: resolve a number of data discrepancies
# there are genes with symbols in the umi_data not in the EnsDb.Hsapiens.v86 
# OR the symbols of the same genes are mismatched 
# there are genes with ensembl_gene_ids in the molecular data not in the EnsDb.Hsapiens.v86
# there seem to be genes with duplicate symbols and different ensembl_gene_ids
# Another discrepancy is that molecular data only includes rows with rowSums > 0, 
# whereas umi includes all irrespective of count

# load human genes
gns = as.data.frame(genes(EnsDb.Hsapiens.v86))
gns$gene_short_name=gns$gene_name
gns$symbol=toupper(gns$symbol)
gns$ensembl_gene_id=row.names(gns)
gene_names_df = gns[, c('ensembl_gene_id', 'symbol')]


for(name in expt_log_pd_human_sn_demult$library[3:length(expt_log_pd_human_sn_demult)]){
    print(name)
    expt_log_row = expt_log_pd_human_sn_demult[expt_log_pd_human_sn_demult$library == name,]
    
    # for now, these files are not used
    # in future iterations, we may want to add these columns to the metadata
    #prob_singlet = read.table(file.path(base_path, name, prob_singlet_path), header = TRUE, sep = "\t")
    #prob_doublet = read.table(file.path(base_path, name, prob_doublet_path), header = TRUE, sep = "\t")

    # load data 
    donor_ids = read.table(file.path(base_path, name, donor_ids_path), header = TRUE, sep = "\t")
    rownames(donor_ids) = donor_ids$cell
    mol_data = readRDS(file.path(base_path, name, 'outs/molecule_info.rds'))
    umi_data = readRDS(file.path(base_path, name, 'outs/filtered_feature_bc_matrix.rds'))
    # for some reason, this step duplicates some row names, which dcGMatrix allows but dataframes do not
    rownames(umi_data) = toupper(rownames(umi_data))
    
    # some umi_data has repeated rownames which causes problems down the road
    # we will sum the umi counts in these rows
    # collect rows with duplicate rownames as a dataframe
    n_occur = data.frame(table(rownames(umi_data)))
    repeated_rownames = n_occur[n_occur$Freq > 1,]$Var1
    duplicated_umi_data = as.data.frame(umi_data[rownames(umi_data) %in% repeated_rownames,])
    duplicated_umi_data['symbol'] = sub("\\.\\d$", "", rownames(duplicated_umi_data))

    # sum across numeric columns
    numeric_cols = setdiff(colnames(duplicated_umi_data), "symbol")
    summed_duped_umi_rows = duplicated_umi_data %>%
        group_by(symbol) %>%
        summarise_at(vars(numeric_cols), sum, na.rm = TRUE) %>%
        ungroup() %>%
        as.data.frame()
    # set rownames to be symbol column, then remove it
    rownames(summed_duped_umi_rows) = summed_duped_umi_rows$symbol
    summed_duped_umi_rows = summed_duped_umi_rows[, !(names(summed_duped_umi_rows) %in% c('symbol'))]

    # Hack
    # each round of the loop will remove ONE of each duplicate row 
    for(i in 1:max(n_occur$Freq)){
        umi_data = umi_data[!rownames(umi_data) %in% repeated_rownames,]
    }

    # finally, replace the deduped rows with the summed rows via an rbind
    umi_data = rbind(umi_data, as(summed_duped_umi_rows, "sparseMatrix"))
    
    # now we will coerce the molecular data into the size of the umi data
    # initially this will have columns 'cell''umi''gem_group''gene''reads''library'
    mol_data_frame = as.data.frame(mol_data[['data']])

    # the $gene column are ints
    # use the `genes` name of the mol data (which is a list of ensemble gene ids) to make that a column in the df
    mol_data_ens_df = as.data.frame(mol_data[['genes']])
    colnames(mol_data_ens_df) = c('ensembl_gene_id')
    mol_data_frame['ensembl_gene_id'] = toupper(mol_data_ens_df$ensembl_gene_id[mol_data_frame$gene])

    # keep only those cells that are in the umi_data
    mol_data_frame$cell = paste0(mol_data_frame$cell, "-1")
    mol_data_frame = mol_data_frame[mol_data_frame$cell %in% colnames(umi_data), ]

    # Group by 'cell' and 'ensembl_gene_id' and sum the number of occurrences
    grouped_df = mol_data_frame %>%
        group_by(cell, ensembl_gene_id) %>%
        summarize(reads = sum(reads), .groups = "drop")

    mol_data_wide = grouped_df %>% 
        pivot_wider(names_from = cell, values_from = reads, values_fill = 0)

    # we need the gene names for these ensemble gene ids
    # NOTE: IT SEEMS THERE ARE SOME ENSEMBLE GENE IDS THAT ARE NOT IN THE GENE NAMES DATAFRAME
    mol_data_wide_joined = merge(mol_data_wide, gene_names_df, by='ensembl_gene_id')

    # find genes with multiple ensemble_gene_ids per symbol
    duplicate_gene_symbols = mol_data_wide_joined %>% 
        group_by(symbol) %>% 
        summarize(n=n(), .groups = "drop") %>% 
        filter(n > 1) %>% 
        arrange(desc(n))

    # isolate the rows corresponding to these duplicated symbols and sum across them
    dupe_rows = mol_data_wide_joined[mol_data_wide_joined$symbol %in% duplicate_gene_symbols$symbol,]
    sum_dupe_rows = dupe_rows %>%
        group_by(symbol) %>%
        summarize(across(-c('ensembl_gene_id'), sum))
    rownames(sum_dupe_rows) = sum_dupe_rows$symbol

    # remove rows with duplicate gene symbols
    mol_data_wide_joined_deduped = mol_data_wide_joined[!mol_data_wide_joined$symbol %in% duplicate_gene_symbols$symbol,]
    # drop the ensembl_gene_id column, which is now redundant
    mol_data_wide_joined_deduped = mol_data_wide_joined_deduped[, !(names(mol_data_wide_joined_deduped) %in% c('ensembl_gene_id'))]
    # now symbol is unambiguous, so we can use it as the rownames
    rownames(mol_data_wide_joined_deduped) = mol_data_wide_joined_deduped$symbol 

    # add back in the summed reads to the mol_data_wide_joined 
    mol_data_wide_joined_deduped_add_back = rbind(mol_data_wide_joined_deduped, sum_dupe_rows)

    # type coercing to data.frame
    umi_df = as.data.frame(umi_data)
    umi_df['symbol'] = rownames(umi_df)
    umi_df = as.data.frame(umi_df[, c('symbol')])
    colnames(umi_df) = c('symbol')

    # finally we want to left join the umi_data to the result above using gene symbol
    mol_data_final = left_join(umi_df, mol_data_wide_joined_deduped_add_back, by='symbol')
    rownames(mol_data_final) = mol_data_final$symbol
    mol_data_final = mol_data_final[, !(names(mol_data_final) %in% c('symbol'))]

    # convert all nans to 0s
    mol_data_final[is.na(mol_data_final)] = 0


    meta_df = createMetaDataFromDCGMatrix(
        expt_log_row,
        umi_data, 
        mol_data_final,
        donor_ids,
        cutoff=DONOR_ASSIGNMENT_CUTOFF)

    data_sce=.myExpSetCreatorFn(inputExpData=umi_data,
                            organism="human",
                            minExpCells=0,
                            inputPdata=meta_df,
                            inputFdata=NULL,
                            addExtraAnno=T,
                            ncores=12)
    saveRDS(data_sce, paste0("~/seq_data/pd/", name, "/data_sce.rds"))
}


[1] "pCalicoPDsHSrSNNURR1iPoold230214"


Warning message:
"Setting row names on a tibble is deprecated."
Warning message in asMethod(object):
"sparse->dense coercion: allocating vector of size 2.6 GiB"


[1] "final mol data:"
[1] 32734 10662
[1] 0
[1] 32734 10662
[1] 0
[1] "Number of MT genes in the dataset: 13 / 13"
[1] "Number of IEG genes in the dataset: 134 / 130"
[1] "Number of OXPHOS genes in the dataset: 180 / 205"
[1] "Number of rRNA genes in the dataset: 1 / 568"
[1] "Number of ribosomal protein genes in the dataset: 104 / 1250"
[1] "pCalicoPDsHSrSNNURR2iPoold230214"


Warning message:
"Setting row names on a tibble is deprecated."
Warning message in asMethod(object):
"sparse->dense coercion: allocating vector of size 2.7 GiB"


[1] "final mol data:"
[1] 32734 11183
[1] 0
[1] 32734 11183
[1] 0
[1] "Number of MT genes in the dataset: 13 / 13"
[1] "Number of IEG genes in the dataset: 134 / 130"
[1] "Number of OXPHOS genes in the dataset: 180 / 205"
[1] "Number of rRNA genes in the dataset: 1 / 568"
[1] "Number of ribosomal protein genes in the dataset: 104 / 1250"
[1] "pCalicoPDsHSrSNNURR3iPoold230214"


...

Warning message in asMethod(object):
"sparse->dense coercion: allocating vector of size 1.9 GiB"
Warning message in asMethod(object):
"sparse->dense coercion: allocating vector of size 1.9 GiB"


[1] "Number of MT genes in the dataset: 13 / 13"
[1] "Number of IEG genes in the dataset: 134 / 130"
[1] "Number of OXPHOS genes in the dataset: 180 / 205"
[1] "Number of rRNA genes in the dataset: 1 / 568"
[1] "Number of ribosomal protein genes in the dataset: 104 / 1250"
[1] "pCalicoPDsHSrSNH8id230921H8"


Warning message:
"Setting row names on a tibble is deprecated."
Warning message in asMethod(object):
"sparse->dense coercion: allocating vector of size 1.8 GiB"


[1] "final mol data:"
[1] 32734  7226
[1] 0
[1] 32734  7226
[1] 0


Warning message in asMethod(object):
"sparse->dense coercion: allocating vector of size 1.8 GiB"
Warning message in asMethod(object):
"sparse->dense coercion: allocating vector of size 1.8 GiB"


In [ ]:
sce_list = list()
for(name in expt_log_pd_human_sn_demult$library){
    print(name)
    fname = file.path("~/seq_data/pd", name, "/data_sce.rds")
    sce = readRDS(fname)
    sce_list = append(sce_list, list(sce))
}
pilot_sce = .mycBindFn(sce_list)
dim(pilot_sce)
saveRDS(pilot_sce, paste0("~/seq_data/pd/pilot_sce.rds"))


In [37]:
# molecular data only includes rows with rowSums > 0
# umi data has all rows, including for genes that were not expressed in the library
# add back in those rows with rowsums == 0 to the mol_data_wide_joined_deduped
umi_rowsums_zero_rownames = toupper(rownames(umi_data))[rowSums(umi_data) == 0]

# Create a matrix of zeros
zero_matrix = matrix(0, nrow=length(umi_rowsums_zero_rownames), ncol=length(colnames(mol_data_wide_joined_deduped_add_back)),
                      dimnames=list(umi_rowsums_zero_rownames, colnames(mol_data_wide_joined_deduped_add_back)))
# Convert matrix to dataframe
rowsums_zero_df = as.data.frame(zero_matrix)

mol_data_wide_joined_deduped_w_zeros = rbind(mol_data_wide_joined_deduped_add_back, rowsums_zero_df)

print("wide data, joined to genes, summed by symbol, 0-rows added back:")
print(dim(mol_data_wide_joined_deduped_w_zeros))
print("UMI data:")
print(dim(umi_data))


[1] "wide data, joined to genes, summed by symbol, 0-rows added back:"
[1] 31754 12991
[1] "UMI data:"
[1] 32738 12990
